<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-23 09:51:14
-------------------
qualified stocks: 95
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.95 L
Current:  1.28 C
-------------------
Today PnL: 42.72 K (0.34%)
Current PnL: -16.82 L (-11.93%)
CY Booked + Current PnL: -12.96 L (-9.19%)
-------------------
Total profit:  3.91 L
Total loss:  -20.73 L
-------------------
Total Booked + Current PnL: 14.02 L (12.29%)
Total Booked PnL: 30.84 L (27.04%)
Curr Year Booked PnL: 3.86 L (3.02%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 80.06 L (62.7%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.41 C (110.58%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1291.83,1526.00,4.23,X-LC,81.05,154671.0,13862.0,11662.0,0.68,9.84,7.54,18.13,161.0,1.19,1.23,22.13,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,0.88,H-LC,87.37,149906.0,24201.0,11798.0,1.44,19.25,7.87,28.63,159.0,2.05,1.19,33.55,AR,ATH,INSURANCE
4,ADANIPORTS,1103.69,1583.00,-1.85,M-LC,89.47,98271.0,19909.0,14122.0,0.93,25.41,14.37,43.43,48.0,1.41,0.78,31.44,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-18.55,X-LC,26.32,90222.0,15100.0,17061.0,0.43,20.10,18.91,42.81,121.0,0.89,0.72,13.15,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-14.39,M-LC,31.58,209218.0,7738.0,19478.0,0.54,3.84,9.31,13.51,60.0,0.40,1.66,16.92,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VBL,500.38,672.28,-14.41,H-LC,1.05,147284.0,-5332.0,57765.0,2.84,-3.49,39.22,34.35,23.0,-0.09,1.17,10.95,AR,ATH,FMCG
0,5PAISA,564.75,810.00,147.60,H-SC,55.79,133608.0,-53324.0,134503.0,2.32,-28.53,100.67,43.43,185.0,-0.40,1.06,27.82,X40N,BTT,FINANCE
50,LTIM,5664.39,7197.33,-5.11,X-LC,92.63,152130.0,-17802.0,63788.0,1.86,-10.48,41.93,27.06,32.0,-0.28,1.21,26.12,X200,ATH,IT
41,INFY,1432.81,2275.00,-12.06,X-LC,69.47,219036.0,19875.0,97186.0,1.83,9.98,44.37,58.78,6.0,0.20,1.74,14.36,X40,BTT,IT
39,INDIGOPNTS,1407.73,2061.00,94.75,H-SC,25.26,127596.0,-46963.0,127966.0,1.69,-26.90,100.29,46.41,41.0,-0.37,1.01,10.65,X40N,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,95.79,65184.0,-48365.0,88416.0,-2.47,-42.59,135.64,35.27,181.0,-0.55,0.52,59.54,XR,NTT,HOTELS
29,GREENPANEL,375.16,537.0,169.38,M-SC,44.21,120243.0,-62835.0,141815.0,-2.07,-34.32,117.94,43.14,109.0,-0.44,0.96,13.11,XY24,NTT,MISC
89,VALIANTORG,512.64,838.0,-330.19,H-SC,64.21,97838.0,-35448.0,120037.0,-2.07,-26.60,122.69,63.47,145.0,-0.30,0.78,64.00,XR,NTT,CHEMICALS
88,VAIBHAVGBL,350.21,521.0,58.68,X-SC,67.89,105922.0,-46069.0,120190.0,-1.82,-30.31,113.47,48.77,57.0,-0.38,0.84,21.67,XR,NTT,JEWELLERY
13,BERGEPAINT,558.54,680.0,-17.52,H-MC,12.63,176886.0,-171.0,38667.0,-1.48,-0.10,21.86,21.75,183.0,-0.00,1.41,26.53,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,2327.09,4911.36,-51.65,H-SC,56.84,126140.0,2804.0,134163.0,-0.53,2.27,106.36,111.05,1.0,0.02,1.00,23.59,AR,ATH,MISC
82,TCS,3888.13,4998.00,-15.84,X-LC,51.58,210600.0,-22688.0,89273.0,0.95,-9.73,42.39,28.55,3.0,-0.25,1.67,8.62,X40,BTT,IT
81,TATAMOTORS,771.32,1065.00,-52.64,M-LC,80.00,210770.0,-15227.0,101275.0,0.27,-6.74,48.05,38.07,4.0,-0.15,1.68,24.08,XY24,NTT,AUTO
9,AWL,342.88,485.00,-69.72,H-MC,4.21,164274.0,-53455.0,143707.0,0.47,-24.55,87.48,41.45,4.0,-0.37,1.31,9.73,XY24,NTT,FMCG
41,INFY,1432.81,2275.00,-12.06,X-LC,69.47,219036.0,19875.0,97186.0,1.83,9.98,44.37,58.78,6.0,0.20,1.74,14.36,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3576.0,-18.55,X-LC,26.32,90222.0,15100.0,17061.0,0.43,20.10,18.91,42.81,121.0,0.89,0.72,13.15,X40,BTT,CHEMICALS
55,PFIZER,4236.33,6318.3,2.30,H-SC,98.95,159328.0,28002.0,36534.0,0.00,21.32,22.93,49.15,94.0,0.77,1.27,31.32,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.0,-1.85,M-LC,89.47,98271.0,19909.0,14122.0,0.93,25.41,14.37,43.43,48.0,1.41,0.78,31.44,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,44.74,M-SC,85.79,150195.0,12617.0,81796.0,0.87,9.17,54.46,68.62,66.0,0.15,1.19,30.21,X40N,NTT,TEXTILES
57,POLYCAB,5005.95,7541.35,6.73,H-MC,65.26,101371.0,16270.0,26833.0,0.25,19.12,26.47,50.65,56.0,0.61,0.81,27.43,X40N,ATH,CABLES
55,PFIZER,4236.33,6318.30,2.30,H-SC,98.95,159328.0,28002.0,36534.0,0.00,21.32,22.93,49.15,94.0,0.77,1.27,31.32,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,-0.23,H-SC,57.89,147271.0,9411.0,28350.0,-0.72,6.83,19.25,27.39,64.0,0.33,1.17,27.64,X40,ATH,FMCG
33,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,82.11,151375.0,462.0,59990.0,1.14,0.31,39.63,40.06,27.0,0.01,1.20,23.82,AR,ATH,AUTO


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-3.56,H-SC,9.47,108441.0,6437.0,38583.0,-0.25,6.31,35.58,44.14,31.0,0.17,0.86,23.58,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,33.16,132873.0,3721.0,83351.0,0.27,2.88,62.73,67.42,57.0,0.04,1.06,29.23,XR,NTT,BANKS
94,WIPRO,243.46,420.00,-10.16,M-LC,46.32,154318.0,3373.0,106078.0,1.20,2.23,68.74,72.51,49.0,0.03,1.23,16.19,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-51.65,H-SC,56.84,126140.0,2804.0,134163.0,-0.53,2.27,106.36,111.05,1.0,0.02,1.00,23.59,AR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,82.11,151375.0,462.0,59990.0,1.14,0.31,39.63,40.06,27.0,0.01,1.20,23.82,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.0,-55.73,L-SC,95.79,65184.0,-48365.0,88416.0,-2.47,-42.59,135.64,35.27,181.0,-0.55,0.52,59.54,XR,NTT,HOTELS
52,MASFIN,326.60,399.5,-23.67,H-SC,60.00,86550.0,-11430.0,33296.0,-0.09,-11.67,38.47,22.32,102.0,-0.34,0.69,25.53,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,-1.85,M-LC,89.47,98271.0,19909.0,14122.0,0.93,25.41,14.37,43.43,48.0,1.41,0.78,31.44,XY24,NTT,MISC
89,VALIANTORG,512.64,838.0,-330.19,H-SC,64.21,97838.0,-35448.0,120037.0,-2.07,-26.60,122.69,63.47,145.0,-0.30,0.78,64.00,XR,NTT,CHEMICALS
86,UJJIVANSFB,52.77,60.0,37.69,M-SC,28.42,119313.0,-23166.0,42690.0,-0.61,-16.26,35.78,13.70,213.0,-0.54,0.95,40.82,X40N,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-847.65,H-SC,100.00,71224.0,-24092.0,68788.0,-0.14,-25.28,96.58,46.89,173.0,-0.35,0.57,40.48,X40N,NTT,MISC
55,PFIZER,4236.33,6318.30,2.30,H-SC,98.95,159328.0,28002.0,36534.0,0.00,21.32,22.93,49.15,94.0,0.77,1.27,31.32,X40,ATH,PHARMA
79,TANLA,917.30,1963.11,-39.81,H-SC,97.89,147609.0,-91806.0,364771.0,0.61,-38.35,247.12,114.01,20.0,-0.25,1.17,35.38,AR,ATH,IT
93,WHIRLPOOL,1167.49,2270.00,-35.54,M-SC,96.84,187866.0,16245.0,145822.0,0.86,9.47,77.62,94.43,28.0,0.11,1.49,38.66,X40,NTT,DURABLES
20,COFFEEDAY,59.14,80.00,-55.73,L-SC,95.79,65184.0,-48365.0,88416.0,-2.47,-42.59,135.64,35.27,181.0,-0.55,0.52,59.54,XR,NTT,HOTELS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.19
1,25,38.66
2,50,68.36


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.06
X40N    13.86
X40     13.82
XY25    13.67
AR      12.32
XR       9.95
X200     1.21
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.74
M-SC    18.22
X-LC    15.76
H-MC     8.43
M-LC     7.80
H-LC     6.97
X-MC     5.67
M-MC     5.63
X-SC     2.82
L-SC     2.33
L-LC     1.18
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,12.13,-1.98,34.86
BANKS,10.95,-9.41,63.07
IT,9.34,-19.63,85.96
MISC,6.56,-31.07,89.05
INSURANCE,6.40,1.67,27.59
FINANCE,6.08,-25.13,84.55
PAINTS,5.54,-17.99,52.90
AUTO,4.53,-9.99,58.04
ELECTRICAL,4.37,-5.63,59.37


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2996758.0,27
AR,1220301.0,13
X40N,1208321.0,17
XR,1100268.0,14
X40,738694.0,11
XY25,474878.0,10
SR,203489.0,2
X200,63788.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2333436.0,21
M-SC,2029573.0,21
X-LC,711804.0,12
M-MC,560780.0,5
H-MC,465069.0,8
X-SC,352643.0,4
M-LC,325360.0,6
X-MC,323526.0,5
L-SC,304933.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      920870.0      6
M-SC       XY24      762498.0      6
H-SC       AR        558871.0      3
           X40N      505424.0      6
M-SC       X40N      460411.0      6
M-MC       XY24      393965.0      3
X-LC       X40       393164.0      5
M-SC       XR        295834.0      4
H-SC       XR        283387.0      4
H-LC       AR        221237.0      4
M-SC       X40       215084.0      2
M-LC       XY24      199804.0      4
H-MC       XY24      182374.0      2
L-SC       XR        153653.0      2
X-MC       XY24      144432.0      2
X-LC       AR        139616.0      2
X-SC       XR        120190.0      1
           SR        119282.0      1
M-SC       XY25      118832.0      1
X-LC       XY25      115236.0      4
X-SC       X40N      113171.0      2
M-LC       XR        106078.0      1
X-MC       X40N      102482.0      2
M-SC       AR         92707.0      1
L-SC       XY24       86864.0      1
M-SC       SR         84207.0      1
M-MC       AR         83464.0      1
           XR         83351.0      1
X-MC       XY25       76612.0      1
H-MC       XY25       72535.0      1
           X40        65562.0      2
H-SC       X40        64884.0      2
L-SC       AR         64416.0      1
X-LC       X200       63788.0      1
H-MC       AR         59990.0      1
           XR         57775.0      1
L-LC       XY25       40942.0      1
H-LC       XY25       31243.0      1
H-MC       X40N       26833.0      1
M-LC       XY25       19478.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 61.0 seconds
